In [1]:
import pandas as pd
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from pathlib import Path
from Measure import measure_final_score
import statistics
from CatEncoder import *
import glob
import pickle

In [2]:
def get_results():
    path = r'/Users/germainherbay/PycharmProjects/Blind/adult/score/directed/sex_recover/10/' # use your path
    all_files = glob.glob(path + "/*.csv")
    first_df = pd.read_csv(all_files[0])
    first_df['name'] = Path(all_files[0]).name
    for filename in all_files[1:]:
        df = pd.read_csv(filename)
        df['name'] = Path(filename).name
        first_df = first_df.append(df)
    first_df['model'] = first_df['model'].replace(['LogisticRegression()','RandomForestClassifier()', 'SVC()', 'MLPClassifier()'],['LR','RF','SVC', 'MLPC'])
    first_df.rename(columns = {'protected_att':'PA'}, inplace = True)
    return first_df

a=get_results()
print(a)

    PA model  recall  accuracy  precision   eod   SPD                 name
0  sex    LR    0.52      0.84       0.75  0.06  0.15  sex_recover10_1.csv
1  sex    RF    0.54      0.83       0.72  0.05  0.13  sex_recover10_1.csv
2  sex   SVC    0.49      0.84       0.79  0.05  0.11  sex_recover10_1.csv
3  sex  MLPC    0.56      0.85       0.77  0.05  0.13  sex_recover10_1.csv
0  sex    LR    0.52      0.84       0.75  0.06  0.15  sex_recover10_2.csv
1  sex    RF    0.54      0.83       0.72  0.07  0.13  sex_recover10_2.csv
2  sex   SVC    0.50      0.84       0.79  0.06  0.11  sex_recover10_2.csv
3  sex  MLPC    0.57      0.85       0.76  0.06  0.14  sex_recover10_2.csv
0  sex    LR    0.52      0.84       0.75  0.05  0.15  sex_recover10_3.csv
1  sex    RF    0.56      0.84       0.73  0.04  0.14  sex_recover10_3.csv
2  sex   SVC    0.50      0.84       0.78  0.05  0.11  sex_recover10_3.csv
3  sex  MLPC    0.49      0.84       0.79  0.05  0.11  sex_recover10_3.csv
0  sex    LR    0.52     